### Intro : isabel
People wanting to learn how to analyze their savings

### still need a disclaimer!!


**Financial Content:** All content provided by Isabel, Luna, Nirantheri, including articles, reports, analyses, opinions, recommendations, and any other financial information, is purely for informational purposes. It should not be construed as financial advice or a recommendation to buy, sell, or hold any financial instrument or investment strategy.

**Not Financial Advice:** Isabel, Luna, Nirantheri are not registered financial advisors, and the information provided should not be considered professional financial advice. It is highly recommended to consult with a qualified financial advisor or investment professional before making any financial decisions based on the information provided by Isabel, Luna, Nirantheri.

**Accuracy of Information:** While Isabel, Luna, Nirantheri aim to provide accurate and current information, we cannot guarantee the accuracy, completeness, or reliability of any content or data provided. Users should independently verify all information before relying on it for financial decision-making purposes.

**Risks of Investing:** Investing in financial markets carries inherent risks, including market volatility, economic factors, geopolitical events, and individual investment performance. Past performance is not indicative of future results, and there are no assurances of investment success.

**No Endorsement:** Any mention of specific financial products, services, companies, or securities within Isabel, Luna, Nirantheri's content does not imply endorsement or recommendation. Users are advised to conduct their own research and due diligence before investing in any financial product or service.

**No Liability:** Isabel, Luna, Nirantheri, their affiliates, employees, or partners shall not be held liable for any damages, losses, or expenses arising from the use of or reliance on the information provided. Users assume full responsibility for their investment decisions and outcomes.

**Changes to Disclaimer:** Isabel, Luna, Nirantheri reserve the right to modify or update this disclaimer at any time without prior notice. Users are responsible for regularly reviewing the disclaimer for any changes.


### using old material : Nirantheri


The code to show the importance of investing vs savings (just draw out each of the ira, 401k, hysa along with no investment to show)
How to use information.md?

### setup

include all packages for setting up (pyfolio, package)

In [ ]:
# all libraries

import pyfolio
import yfinance as yf
import numpy as np
import scipy
import ipywidgets
import pandas as pd
from scipy.optimize import minimize

### Choosing stocks for a portfolio: Nirantheri

Give them a list of 20-30 common stocks, etfs, and mutual funds (can use checkbox widget)
ipywidgets: 
[checkbox](https://ipywidgets.readthedocs.io/en/7.x/examples/Widget%20List.html#Checkbox)
[select multiple](https://ipywidgets.readthedocs.io/en/7.x/examples/Widget%20List.html#SelectMultiple)
ex.
- S&P 500
- Dow
- Nasdaq
- Apple
- Tesla
- Google
- Gold??
- etc

### Picking your own breakdown of stocks : Nirantheri

use another widget, maybe with a slider (tbd)

[setup using the above code chunk](https://ipywidgets.readthedocs.io/en/7.x/examples/Widget%20List.html#Tabs)

### pulling the yf data: Luna

Assume that the list of stocks will come in as a set of stock codes so you can write a loop to get each of them

In [ ]:
#pulled a dictionary of different stocks, keys are tickers? i cant think of a better way to store them because the list is variable so its a bit of an issue

testList = ['AAPL', 'MSFT', 'GOOG', 'NVDA', '^GSPC', '^DJI', '^IXIC']
testString = ' '.join(testList)

tickersPull = yf.Tickers(testString)

#in the dictionary dataframes, each ticker is indexed by its ticker as seen in the testList
dataframes = {}
for x in testList:
    ticker = yf.Ticker(x)
    dataframes[x] = ticker.history(period='1mo', start='2015-01-01')
    print(dataframes[x])



### check strength of portfolio : Isabel
pyfolio: https://www.pyquantnews.com/the-pyquant-newsletter/create-beautiful-strategy-tear-sheets-pyfolio-reloaded
can also use quantstats



pyfolio: get the tear sheets--> then use pandas to pull specific metrics (annual return, etc) and then have a md cell which breaks down what each of them means

### montecarlo simulations : tbd

uses yahoo finance package and others to create montecarlo simulation for modeling

montecarlo https://medium.com/analytics-vidhya/monte-carlo-simulations-for-predicting-stock-prices-python-a64f53585662

### optimizing the set of stocks : luna

use scipy

optimizing your investment: https://medium.com/@ethan.duong1120/python-powered-portfolio-optimization-achieving-target-returns-through-weight-optimization-fc5163e5c9c6


In [ ]:
testList = ['AAPL', 'MSFT', 'GOOG', 'TSLA', 'NFLX', 'SBUX', '^GSPC', '^DJI', '^IXIC']

# pull the first stock's data; drop all unnecessary columns
ticker = yf.Ticker(testList[0])
df = ticker.history(interval = '1d', start = '2015-01-01', end = '2025-04-01')
df.drop(columns=['High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits'], inplace=True)
df.rename({'Open' : 'AAPL'}, inplace=True)

# for every other ticker, add its open value to the existing dataframe
for x in testList[1:]:
    ticker = yf.Ticker(x)
    data = ticker.history(interval = '1d', start = '2015-01-01', end = '2025-04-01')
    df.insert(len(df.columns), x, data["Open"]) # for whatever reason, this line wont work

#calculate percent returns for each day of each stock
returns_df = df.pct_change(1).dropna()

#operationalize determining portfolio returns
def getPfReturn(weights):
    """
    return is annualized expected return of portfolio
    """
    expRetPortfolio = np.dot(np.transpose(weights), returns_df.mean()) * 250
    return expRetPortfolio

# start with stocks at equal weights
numStocks = len(returns_df.columns)
initialWeight = [1/numStocks] * numStocks

# return goal?
targetReturn = .4

# bounds the percentage of each stock we can hold (between 0 and 100%)
bounds = tuple((0,1) for i in range(numStocks))

# ensures the sum of all stock weights is 100% (or 1) in first constraint
# sets goal of minimize function to hit targetReturn
constraints = ({'type' : 'eq', 'fun' : lambda w : np.sum(w) - 1},
               {'type' : 'eq', 'fun' : lambda x : x.dot(returns_df.mean()) * 250 - targetReturn})

# can we reach our goal with these stocks??
results = minimize(fun=getPfReturn, x0=initialWeight, bounds=bounds, constraints=constraints)

#output
print(results)

optimizedResults = pd.DataFrame(results['x'])

# print our optimized results
getPfReturn(weights=results["x"])
optimizedResults.index = returns_df.columns
print(optimizedResults)

### rechecking the strength of your portfolio : isabel's

go back to pyfolio--> will integrate after we recheck the working code